In [25]:
import os
import json
import sys
mocks_dir = os.path.join(os.path.abspath(''), 'mocks')
sys.path.insert(0, mocks_dir + '/../../classes/')
#print(mocks_dir + '/../../classes/')
from patent import Patent

In [26]:
data_path = os.path.join(mocks_dir, 'validation_data.json')
data_file = open(data_path)
data = json.load(data_file)

In [27]:
def print_result(patent_number, key, expected, extracted):
    print(key + " is incorrect for patent id " + patent_number)
    print("Extracted: " + extracted)
    print("Expected: " + expected)

In [28]:
#patent id, expected is subset of extracted, extracted is subset of expected, expected, extracted
import pandas as pd



In [30]:
epitope_results =  pd.DataFrame(columns=['patent_id', 'expected_is_subset', 'extracted_is_subset', 'expected_residues', 'extracted_residues'])

for datum in data:
    pat_id = datum['patentNumber']
    patent_xml_path = os.path.join(mocks_dir, pat_id + '.xml')
    patent = Patent(patent_xml_path, pat_id)
    
    for key in datum:
        if type(datum[key]) == str:
            if datum[key].lower() != patent.__dict__[key].lower(): 
                print_result(datum['patentNumber'], key, datum[key], patent.__dict__[key])
        elif key == 'mentionedResidues':
            extracted_res = set()
            for a in patent.__dict__[key]:
                for b in a['claimedResidues']:
                    extracted_res.add(b)
            #parsed_epitopes = list(res)
            extracted_res = set(sorted(extracted_res, key=lambda e: int(e)))
            expected_res = set(sorted(datum[key], key=lambda e: int(e)))
            exp_sub = expected_res.issubset(extracted_res)
            ext_sub = extracted_res.issubset(expected_res)
            epitope_results = epitope_results.append(
                {'patent_id': pat_id, 'expected_is_subset': exp_sub, 'extracted_is_subset': ext_sub, 
                'expected_residues': expected_res, 'extracted_residues': extracted_res}, ignore_index=True)
            
        else: 
            if datum[key] != patent.__dict__[key]:
                print_result(datum['patentNumber'], key, str(datum[key]), str(patent.__dict__[key]))


In [31]:
epitope_results

,patent_id,expected_is_subset,extracted_is_subset,expected_residues,extracted_residues
0,US8889144B2,True,True,{},{}
1,US10787484B2,True,False,"{364, 442, 241, 444, 339, 343, 443}","{342, 444, 339, 341, 240, 367, 1, 340, 1369, 2..."
